In [30]:
from notebookutils import mssparkutils
from pyspark.sql.functions import lit

In [32]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [33]:
año = name[0:4]
mes = name[4:6]

In [34]:
account_name = "alexstorage0003"
container_name = "datalake"
relative_path_in = "bronze/car_database"
relative_path_out = "silver/car_database"

adls_path_in = 'abfss://%s@%s.dfs.core.windows.net/%s'%(container_name, account_name, relative_path_in)
adls_path_output = 'abfss://%s@%s.dfs.core.windows.net/%s'%(container_name, account_name, relative_path_out)

In [35]:
dataframe = spark.read.option('header', 'false') \
.option('delimiter',',') \
.csv(adls_path_in + f'/{name}')

In [36]:
dataframe = dataframe.withColumnRenamed("_c0","placa")\
                        .withColumnRenamed("_c1","marca")\
                        .withColumnRenamed("_c2","modelo")\
                        .withColumnRenamed("_c3","id_cliente")\
                        .withColumnRenamed("_c4","comuna")\
                        .withColumnRenamed("_c5","region")\
                        .withColumnRenamed("_c6","sexo")\
                        .withColumnRenamed("_c7","actividad")\
                        .withColumnRenamed("_c8","valor_vehiculo")\
                        .withColumnRenamed("_c9","fecha_transferencia")\
                        .withColumnRenamed("_c10","color")\
                        .withColumnRenamed("_c11","edad")\
                        .withColumnRenamed("_c12","vigencia")\
                        .select("*", lit(f"{año}").alias("año"), lit(f"{mes}").alias("mes"))

In [37]:
dataframe.write\
    .mode('overwrite')\
    .format("avro")\
    .partitionBy('año', 'mes')\
    .save(adls_path_output)

In [39]:
mssparkutils.notebook.exit(1) 